In [9]:
import os
import sys
# current = os.path.dirname(os.path.realpath("__file__"))
# parent = os.path.dirname(current)
# sys.path.append(parent)
from pathlib import Path # if you haven't already done so
file = Path(os.path.dirname(os.path.abspath("__file__"))).resolve()
parent, root = file.parent, file.parents[1]
sys.path.append(str(root))

import torch
from dtmgp.models import SCNN, DtmgpAdditive, DtmgpSparsegrid
from dtmgp.utils.sparse_activation.design_class import HyperbolicCrossDesign
from dtmgp.kernels.laplace_kernel import LaplaceProductKernel

In [10]:
torch.manual_seed(10)  # To reproduce results freeze a seed

# Set up the compute device: CPU or Cuda, if you have multiple GPUs, you can choose one using <cuda:0>
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
print(f"Using {device} device")

Using cuda device


In [11]:
# A function to count the number of parameters of a network
def parameters(model):
    all_parameters=list(model.parameters())
    layer_parameters=[len(i) for i in all_parameters] 
    print(layer_parameters) 

    # we have layer weights, followed by biases
    # sum up list 
    print("Total number of parameters in the network",sum(layer_parameters))
    return sum(layer_parameters)

In [12]:
scnn = SCNN().to(device)
parameters(scnn)

addmodel = DtmgpAdditive(input_dim=784,
                              output_dim=10,
                              design_class=HyperbolicCrossDesign,
                              kernel=LaplaceProductKernel(1.)).to(device)
parameters(addmodel)

sgmodel = DtmgpSparsegrid(input_dim=784,
                          output_dim=10,
                          design_class=HyperbolicCrossDesign,
                          kernel=LaplaceProductKernel(1.)).to(device)
parameters(sgmodel)

[8, 8, 8, 8, 16, 16, 16, 16, 1000, 1000, 1000, 1000, 10, 10, 10, 10]
Total number of parameters in the network 4136
[100, 100, 100, 100, 64, 64, 64, 64, 10, 10, 10, 10]
Total number of parameters in the network 696
[8, 8, 8, 8, 8, 8, 8, 8, 10, 10, 10, 10]
Total number of parameters in the network 104


104